In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.0 MB/s eta 0:00:00


In [ ]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

# Load the pre-trained tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# Load the pre-trained model for sequence classification
model = GPT2ForSequenceClassification.from_pretrained('gpt2')

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
texts = ["I loved the movie. It was great!",
         "The food was terrible.",
         "The weather is okay.",
         "The generative AI class began slow but has picked up."]
sentiments = ["positive", "negative", "neutral", "positive"]
instructions = ["Analyze the sentiment of the text and identify if it is positive.",
                "Analyze the sentiment of the text and identify if it is negative.",
                "Analyze the sentiment of the text and identify if it is neutral.",
                "Analyze the sentiment of the text and identify if it is positive."]

In [ ]:
sentiment_dict = {'positive': 1, 'negative': 2, 'neutral': 3}
sentiment_labels= [sentiment_dict[sentiment] for sentiment in sentiments]



0

In [ ]:
# Tokenize the texts, sentiments, and instructions
encoded_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
encoded_instructions = tokenizer(instructions, padding=True, truncation=True, return_tensors='pt')

# Extract input IDs, attention masks, and instruction IDs
input_ids = encoded_texts['input_ids']
attention_mask = encoded_texts['attention_mask']
instruction_ids = encoded_instructions['input_ids']

In [ ]:
import torch

# Concatenate instruction IDs with input IDs and adjust attention mask
input_ids = torch.cat([instruction_ids, input_ids], dim=1)
attention_mask = torch.cat([torch.ones_like(instruction_ids), attention_mask], dim=1)
# sentiment_labels = [sentiments.index(sentiment) for sentiment in sentiments]


In [ ]:
input_ids.type()

'torch.LongTensor'

In [ ]:
attention_mask.type()

'torch.LongTensor'

In [ ]:
x  = torch.tensor(sentiment_labels)

In [ ]:
x.shape

torch.Size([4])

In [ ]:
x

tensor([0, 1, 2, 0])

In [ ]:
input_ids.shape

torch.Size([4, 26])

In [ ]:
attention_mask.shape

torch.Size([4, 26])

In [ ]:
x.shape

torch.Size([4])

In [ ]:
import torch.optim as optim

# Define the optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

# Fine-tune the model
num_epochs = 3
for epoch in range(num_epochs):
    optimizer.zero_grad()
    sentiment_labels = torch.tensor(sentiment_labels)

    outputs = model(input_ids, attention_mask=attention_mask, labels=sentiment_labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

IndexError: ignored